In [2]:
import sys
import warnings
import concurrent.futures
import queue
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1)
tol = 1e-6
max_iter = 100



np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)


###################################################################################

def share(input):
    #print(input)
    np.random.seed(1)
    x = 0.001 * np.random.random((np.shape(input)))
    #x = np.round(np.random.uniform(-2,2,np.shape(input)),7)
    # print(x)

    np.random.seed(1)
    y = 0.001 * np.random.random((np.shape(input)))
    #y = np.round(np.random.uniform(-2,2,np.shape(input)),7)

    z = input - x - y

    return x, y, z

def sharey(input):
    np.random.seed(1)
    x = np.random.randint(0 , 2 , np.shape(input))

    np.random.seed(1)
    y = np.random.randint(0 , 2 , np.shape(input))

    z = input - x - y

    return x, y, z

###################################################################################

# sizex = 100
# sizey = 20

# rs12 = np.random.RandomState(1)
# X_train = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
# X_train1 , X_train2 , X_train3 = share(X_train)


# rs13 = np.random.RandomState(2)
# y_train = rs13.randint(0, 2, (sizex, 1))
# y_train1 , y_train2 , y_train3 = sharey(y_train)


###################################################################################



data = pd.read_csv('pcs.csv')

Y1 = data.iloc[:, -1:]
y = Y1.values.reshape(np.shape(data)[0], 1)

data.drop(data.columns[len(data.columns) - 1], axis=1, inplace=True)
X = np.array(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

X_train1 , X_train2 , X_train3 = share(X_train)
y_train1 , y_train2 , y_train3 = sharey(y_train)

###################################################################################

def CP1(beta):

    def mul_sharemind(first, second):

        np.random.seed(1)
        r12 = np.random.uniform(0.0000000001, 10**(-40), np.shape(first))
        np.random.seed(2)
        j12 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))
        #print(r12)

        q1.put(r12)
        q1.put(j12)

        r31 = q3.get()
        j31 = q3.get()

        XA11 = first + r12 - r31

        XB11 = second + j12 - j31

        q11.put(XA11)
        q11.put(XB11)

        XA33 = q33.get()
        XB33 = q33.get()

        W1 = XA11 * XB11 + XA11 * XB33 + XA33 * XB11
        #print(W1)


        return W1

    def mul_sharemind_matrix_full(first, second):
        #print(first)

        np.random.seed(1)
        r12 = 0.01 * np.random.random((np.shape(first)))
        #print(r12)
        np.random.seed(1)
        j12 = 0.01 * np.random.random((np.shape(second)))

        q1.put(r12)
        q1.put(j12)

        r31 = q3.get()
        j31 = q3.get()

        XA11 = first + r12 - r31
        #print(XA11)

        XB11 = second + j12 - j31
        #print(j31)

        q11.put(XA11)
        q11.put(XB11)

        XA33 = q33.get()
        XB33 = q33.get()

        W1 = XA11.dot(XB11) + XA11.dot(XB33) + XA33.dot(XB11)

        return W1

    def mul_sharemind_matrix_direct(first, second):
        #print(second)

        np.random.seed(1)
        r12 = np.random.uniform(0.0000000001, 10**(-40), np.shape(first))
        np.random.seed(1)
        j12 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))

        q1.put(r12)
        q1.put(j12)

        r31 = q3.get()
        j31 = q3.get()

        XA11 = first + r12 - r31
        XB11 = second + j12 - j31

        q11.put(XA11)
        q11.put(XB11)

        XA33 = q33.get()
        XB33 = q33.get()

        W1 = np.multiply(XA11, XB11) + np.multiply(XA11, XB33) + np.multiply(XA33, XB11)
        return W1

#     np.random.seed(1)
#     X = np.concatenate((XA1, XB1), axis=0)
    
    X = X_train1
    
    np.random.seed(1)
    XB = mul_sharemind_matrix_full(X, beta)
    #print(XB)


    Pi_temp = np.exp(-XB)
    #print(Pi_temp)

    Pi_temp11, Pi_temp12, Pi_temp13 = share(Pi_temp)



    q12.put(Pi_temp12)
    q13.put(Pi_temp13)

    Pi_temp21 = q31.get()
    Pi_temp31 = q21.get()

    e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp11, Pi_temp21)
    #print('p1 \n',e_to_the_XB_temp)

    e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp31) + 0.4

    np.random.seed(1)
    c_0_1 = np.random.uniform(0.01, 10**(-5), np.shape(e_to_the_XB))

    Xs = c_0_1 * 1 / 3
    Ms = mul_sharemind_matrix_direct(c_0_1 * 1 / 3, e_to_the_XB)
    #print(Xs)
    #print(Ms)


    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
        #print(Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        #print(Xs)
        Ms = M_s_t

#     y = np.concatenate((YA1, YB1), axis=0)
    y = y_train1
    grad_temp = y - X_s_t

    #print('grad_temp \n',grad_temp)
    #print('X.T',X.T)

    Gradient = mul_sharemind_matrix_full(X.T, grad_temp)
    #print('Gradient',Gradient)

    W = np.diag(mul_sharemind_matrix_direct(X_s_t, (1 / 3 - X_s_t))[:, 0])
    #print(W)

    Hessian_temp1 = mul_sharemind_matrix_full(X.T, W)
    #print(W)
    H_temp = mul_sharemind_matrix_full(Hessian_temp1, X)
    #print(H_temp)
    B = np.max(np.linalg.eigvals(H_temp).real)
    #B = np.max(np.linalg.eigvals(H_temp))


    # print('Hessian_temp1 \n' , Hessian_temp1)
    # print('H_temp1 \n' , H_temp)
    #print('B1 \n' , B)
    #print('np.linalg.eigvals(H_temp) \n' , np.linalg.eigvals(H_temp).real)

#     np.random.seed(1)
#     c_0_1 = np.random.uniform(0.0000000001, 10**(-40), (1, 1))
#     Xs = c_0_1 * 1 / 3
#     #print(Xs)

#     Ms = c_0_1 * B
#     #print(Ms)

#     for x in range(0, 40):
#         Ms_Xs = mul_sharemind(Xs, Ms)
#         #print(Ms_Xs)
#         mul_2 = mul_sharemind([[2 / 3]], Xs)
#         #print(mul_2)
#         X_s_t = mul_2 - Ms_Xs

#         Ms_Ms = mul_sharemind(Ms, Ms)
#         Ms_2 = mul_sharemind([[2 / 3]], Ms)
#         M_s_t = Ms_2 - Ms_Ms

#         Xs = X_s_t
#         Ms = M_s_t

#     Inve_eigen = X_s_t

    #Inve_eigen1 = 1/B
    #
    #print('Inve_eigen: \n' , Inve_eigen)
    #print('Inve_eigen1: \n' , Inve_eigen1)


    c = 0.0000002
    q = np.size(H_temp, 0)
    I = np.identity(q)
    Xs = mul_sharemind(c, I / 3)
    Ms = mul_sharemind(c, H_temp)

    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_full(Xs, Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_full(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        #print('X_s_t: \n', X_s_t)

        Ms = M_s_t

    Hessian_Invers = X_s_t
    #print('Hessian_Invers: \n' ,Hessian_Invers)

    Beta = mul_sharemind_matrix_full(Hessian_Invers, Gradient)
    #print('Beta \n',Beta)

    return Beta


###################################################################################

def CP2(beta):
    def mul_sharemind(first, second):

        # np.random.seed(2)
        # r23 = np.random.uniform(0.001, 10 ** (-6), np.shape(first))
        # # np.random.random((np.shape(first)))
        #
        # np.random.seed(3)
        # j23 = np.random.uniform(0.001, 10 ** (-6), np.shape(second))
        #
        #
        np.random.seed(1)
        r23 = np.random.uniform(0.0000000001, 10**(-40), np.shape(first))

        np.random.seed(13)
        j23 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))
        #print(j23)


        q2.put(r23)
        q2.put(j23)

        r12 = q1.get()
        j12 = q1.get()

        XA22 = first + r23 - r12
        XB22 = second + j23 - j12

        q22.put(XA22)
        q22.put(XB22)

        XA11 = q11.get()
        XB11 = q11.get()

        W2 = XA22 * (XB22) + XA22 * (XB11) + XA11 * (XB22)
        return W2

    def mul_sharemind_matrix_full(first, second):
        #print(second)
        np.random.seed(1)  # set the seed
        r23 = 0.01 * np.random.random((np.shape(first)))

        np.random.seed(1)  # set the seed
        j23 = 0.01 * np.random.random((np.shape(second)))

        q2.put(r23)
        q2.put(j23)

        r12 = q1.get()
        j12 = q1.get()

        XA22 = first + r23 - r12
        XB22 = second + j23 - j12

        q22.put(XA22)
        q22.put(XB22)

        XA11 = q11.get()
        XB11 = q11.get()

        W2 = XA22.dot(XB22) + XA22.dot(XB11) + XA11.dot(XB22)
        return W2

    def mul_sharemind_matrix_direct(first, second):

        # r23 = 0.1 * np.random.random((np.shape(first)))
        #
        # np.random.seed(1)  # set the seed
        # j23 = 0.1 * np.random.random((np.shape(second)))

        np.random.seed(1)  # set the seed
        r23 = np.random.uniform(00.0000000001, 10**(-40), np.shape(first))
        # np.random.random((np.shape(first)))

        np.random.seed(1)
        j23 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))


        q2.put(r23)
        q2.put(j23)

        r12 = q1.get()
        j12 = q1.get()

        XA22 = first + r23 - r12
        XB22 = second + j23 - j12

        q22.put(XA22)
        q22.put(XB22)

        XA11 = q11.get()
        XB11 = q11.get()

        W2 = np.multiply(XA22, XB22) + np.multiply(XA22, XB11) + np.multiply(XA11, XB22)
        return W2

#     np.random.seed(1)
#     X = np.concatenate((XA2, XB2), axis=0)

    X = X_train2
    #print(X)
    np.random.seed(1)
    XB = mul_sharemind_matrix_full(X, beta)
    #print(XB)

    Pi_temp = np.exp(-XB)
    #print(Pi_temp)

    Pi_temp21, Pi_temp22, Pi_temp23 = share(Pi_temp)



    q21.put(Pi_temp21)
    q23.put(Pi_temp23)

    Pi_temp12 = q12.get()
    Pi_temp32 = q32.get()

    e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp22, Pi_temp12)
    #print(e_to_the_XB_temp)

    e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp32) + 0.3
    #print('p2 \n', e_to_the_XB)

    np.random.seed(1)
    c_0_2 = np.random.uniform(0.0000000001, 10**(-40), np.shape(e_to_the_XB))
    Xs = c_0_2 * 1 / 3
    Ms = mul_sharemind_matrix_direct(c_0_2 * 1 / 3, e_to_the_XB)

    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        Ms = M_s_t

#     y = np.concatenate((YA2, YB2), axis=0)
    y = y_train2
    grad_temp = y - X_s_t
    Gradient = mul_sharemind_matrix_full(X.T, grad_temp)

    W = np.diag(mul_sharemind_matrix_direct(X_s_t, (1 / 3 - X_s_t))[:, 0])
    Hessian_temp1 = mul_sharemind_matrix_full(X.T, W)
    H_temp = mul_sharemind_matrix_full(Hessian_temp1, X)
    #     print('H_temp2 \n' , H_temp)

    # print('t ',is_pos_def(H_temp),'\n',H_temp)

    #     H = mul_sharemind_matrix_full(X.T , X)
    #     H_temp = 1/4*H

    B = np.max(np.linalg.eigvals(H_temp).real)
    #B = np.max(np.linalg.eigvals(H_temp))

    #     B = np.max(np.linalg.eigvals(H_temp))

    # print('H_temp2 \n' , H_temp)
    # print('B2 \n' , B)
    #     print('np.linalg.eigvals(H_temp) \n' , np.linalg.eigvals(H_temp))

#     np.random.seed(1)
#     c_0_2 = np.random.uniform(00.0000000001, 10**(-40), (1, 1))
#     Xs = c_0_2 * 1 / 3
#     Ms = c_0_2 * B

#     for x in range(0, 40):
#         Ms_Xs = mul_sharemind(Xs, Ms)
#         mul_2 = mul_sharemind([[2 / 3]], Xs)
#         X_s_t = mul_2 - Ms_Xs

#         Ms_Ms = mul_sharemind(Ms, Ms)
#         Ms_2 = mul_sharemind([[2 / 3]], Ms)
#         M_s_t = Ms_2 - Ms_Ms

#         Xs = X_s_t
#         Ms = M_s_t

#     Inve_eigen = X_s_t

    #print('Inve_eigen2: \n' , Inve_eigen)
    #
    # Inve_eigen = 1/B

    c = 0.0000002
    q = np.size(H_temp, 0)
    I = np.identity(q)
    Xs = mul_sharemind(c, I / 3)
    Ms = mul_sharemind(c, H_temp)

    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_full(Xs, Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_full(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        Ms = M_s_t

    Hessian_Invers = X_s_t
    #     print('Hessian_Invers \n',Hessian_Invers)

    Beta = mul_sharemind_matrix_full(Hessian_Invers, Gradient)
    #     print('Beta \n',Beta)

    return Beta


###################################################################################

def CP3(beta):
    def mul_sharemind(first, second):

        # np.random.seed(3)
        # r31 = np.random.uniform(0.001, 10 ** (-6), np.shape(first))
        #
        # np.random.seed(3)
        # j31 = np.random.uniform(0.001, 10 ** (-6), np.shape(second))

        np.random.seed(1)
        r31 = np.random.uniform(0.0000000001, 10**(-40), np.shape(first))
        np.random.seed(1)
        j31 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))

        q3.put(r31)
        q3.put(j31)

        r23 = q2.get()
        j23 = q2.get()

        XA33 = first + r31 - r23
        XB33 = second + j31 - j23

        q33.put(XA33)
        q33.put(XB33)

        XA22 = q22.get()
        XB22 = q22.get()

        W3 = XA33 * (XB33) + XA33 * (XB22) + XA22 * (XB33)
        return W3

    def mul_sharemind_matrix_full(first, second):

        np.random.seed(1)
        r31 = 0.01 * np.random.random((np.shape(first)))

        np.random.seed(1)
        j31 = 0.01 * np.random.random((np.shape(second)))

        q3.put(r31)
        q3.put(j31)

        r23 = q2.get()
        j23 = q2.get()

        XA33 = first + r31 - r23
        XB33 = second + j31 - j23

        q33.put(XA33)
        q33.put(XB33)

        XA22 = q22.get()
        XB22 = q22.get()

        W3 = XA33.dot(XB33) + XA33.dot(XB22) + XA22.dot(XB33)
        return W3

    def mul_sharemind_matrix_direct(first, second):


        # r31 = 0.1 * np.random.random((np.shape(first)))
        #
        # np.random.seed(1)
        # j31 = 0.1 * np.random.random((np.shape(second)))

        np.random.seed(1)
        r31 = np.random.uniform(0.0000000001, 10**(-40), np.shape(first))
        np.random.seed(1)
        j31 = np.random.uniform(0.0000000001, 10**(-40), np.shape(second))


        q3.put(r31)
        q3.put(j31)

        r23 = q2.get()
        j23 = q2.get()

        XA33 = first + r31 - r23
        XB33 = second + j31 - j23

        q33.put(XA33)
        q33.put(XB33)

        XA22 = q22.get()
        XB22 = q22.get()

        W3 = np.multiply(XA33, XB33) + np.multiply(XA33, XB22) + np.multiply(XA22, XB33)
        return W3


#     X = np.concatenate((XA3, XB3), axis=0)
    #print(X)
    
    X = X_train3
    XB = mul_sharemind_matrix_full(X, beta)
    #print(XB)

    Pi_temp = np.exp(-XB)
    #print(Pi_temp)

    Pi_temp31, Pi_temp32, Pi_temp33 = share(Pi_temp)

    q31.put(Pi_temp31)
    q32.put(Pi_temp32)

    Pi_temp13 = q13.get()
    Pi_temp23 = q23.get()

    e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp33, Pi_temp13)
    e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp23) + 0.3
    #print('p3 \n', e_to_the_XB)

    np.random.seed(1)
    c_0_3 = np.random.uniform(0.001, 10 ** (-40), np.shape(e_to_the_XB))
    Xs = c_0_3 * 1 / 3
    Ms = mul_sharemind_matrix_direct(c_0_3 * 1 / 3, e_to_the_XB)

    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        Ms = M_s_t

#     y = np.concatenate((YA3, YB3), axis=0)
    y = y_train3
    grad_temp = y - X_s_t
    Gradient = mul_sharemind_matrix_full(X.T, grad_temp)

    W = np.diag(mul_sharemind_matrix_direct(X_s_t, (1 / 3 - X_s_t))[:, 0])
    Hessian_temp1 = mul_sharemind_matrix_full(X.T, W)
    H_temp = mul_sharemind_matrix_full(Hessian_temp1, X)
    # print('t ',is_pos_def(H_temp),'\n',H_temp)
    # print('H_temp3 \n' , H_temp)

    B = np.max(np.linalg.eigvals(H_temp).real)
    #B = np.max(np.linalg.eigvals(H_temp))

    #     B = np.max(np.linalg.eigvals(H_temp))

    # print('B3 \n' , B)
    #     print('np.linalg.eigvals(H_temp) \n' , np.linalg.eigvals(H_temp))
    #
#     np.random.seed(1)
#     c_0_3 = np.random.uniform(0.0000000001, 10**(-40), (1, 1))
#     Xs = c_0_3 * 1 / 3
#     Ms = c_0_3 * B

#     for x in range(0, 40):
#         Ms_Xs = mul_sharemind(Xs, Ms)
#         mul_2 = mul_sharemind([[2 / 3]], Xs)
#         X_s_t = mul_2 - Ms_Xs

#         Ms_Ms = mul_sharemind(Ms, Ms)
#         Ms_2 = mul_sharemind([[2 / 3]], Ms)
#         M_s_t = Ms_2 - Ms_Ms

#         Xs = X_s_t
#         Ms = M_s_t

#     Inve_eigen = X_s_t

    #print('Inve_eigen3: \n' , Inve_eigen)

    # Inve_eigen = 1/B
    #print(Inve_eigen)

    c = 0.0000002
    q = np.size(H_temp, 0)
    I = np.identity(q)
    Xs = mul_sharemind(c, I / 3)
    Ms = mul_sharemind(c, H_temp)

    for x in range(0, 20):
        Ms_Xs = mul_sharemind_matrix_full(Xs, Ms)
        mul_2 = mul_sharemind([[2 / 3]], Xs)
        X_s_t = mul_2 - Ms_Xs

        Ms_Ms = mul_sharemind_matrix_full(Ms, Ms)
        Ms_2 = mul_sharemind([[2 / 3]], Ms)
        M_s_t = Ms_2 - Ms_Ms

        Xs = X_s_t
        Ms = M_s_t

    Hessian_Invers = X_s_t
    #     print('Hessian_Invers \n',Hessian_Invers)

    Beta = mul_sharemind_matrix_full(Hessian_Invers, Gradient)
    #     print('Beta \n',Beta)

    return Beta


###################################################################################

q1 = queue.Queue()
q2 = queue.Queue()
q3 = queue.Queue()

q11 = queue.Queue()
q22 = queue.Queue()
q33 = queue.Queue()

q12 = queue.Queue()
q13 = queue.Queue()

q21 = queue.Queue()
q23 = queue.Queue()

q31 = queue.Queue()
q32 = queue.Queue()

###################################################################################


#beta_old, beta = np.zeros((np.size(XA + XB, 1), 1)), np.zeros((np.size(XA + XB, 1), 1))


def sharemind(beta):
    beta1, beta2, beta3 = share(beta)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        f1 = executor.submit(CP1, beta1)
        f2 = executor.submit(CP2, beta2)
        f3 = executor.submit(CP3, beta3)

        Total = (f1.result() + f2.result() + f3.result())

        #         print('Beta1, round' , iter_count , '\n', f1.result(), '\n')

        #         print('Beta2, round' , iter_count, '\n', f2.result(), '\n')

        #         print('Beta3, round' , iter_count, '\n', f3.result(), '\n')

        #         print('Tot, round' , iter_count, '\n', Total, '\n')

        #         print(is_pos_def(Total))

        #         print('######################' ,'\n')

        return Total

    return ret

# print('total \n', sharemind(beta))

import atexit
from time import time, strftime, localtime
from datetime import timedelta

# def secondsToStr(elapsed=None):
#     if elapsed is None:
#         return strftime("%Y-%m-%d %H:%M:%S", localtime())
#     else:
#         return str(timedelta(seconds=elapsed))

# def log(s, elapsed=None):
#     line = "="*40
#     print(line)
#     print(secondsToStr(), '-', s)
#     if elapsed:
#         print("Elapsed time:", elapsed)
#     print(line)
#     print()

# def endlog():
#     end = time()
#     elapsed = end-start
#     log("End Program", secondsToStr(elapsed))


def check_coefs_convergence(beta_old, beta_new, tol, iters):
    coef_change = np.abs(beta_old - beta_new)
    # print('coef_change\n',coef_change , '\n')

    return not (np.any(coef_change > tol) & (iters < max_iter))


def newton_step(curr):
    step = sharemind(curr)

    beta = curr + step

    #print('\n', 'BetaTotal:', iter_count ,'\n', beta ,'\n')

    return beta


start = time()
beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))

iter_count = 0

coefs_converged = False

while not coefs_converged:
    beta_old = beta

    iter_count += 1

    beta = newton_step(beta)

    # print('\n', 'BetaTotal:', iter_count, '\n', beta, '\n')

    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)

end = time()
elapsed = end - start
print('Elapsed :', end - start)    
    
print('\n', 'BetaTotal:', iter_count, '\n', beta.real, '\n')
# endlog()



Elapsed : 0.42939114570617676

 BetaTotal: 14 
 [[-0.05175582]
 [ 0.27593971]
 [-1.67830874]
 [-0.99966849]
 [-0.16254882]
 [ 0.66808964]
 [ 0.03313619]
 [-0.01507897]
 [ 0.52231388]] 

